In [ ]:
# default_exp nlp

# Annotating Kanji with Hiragana - Japanese Text Processing with PyKNP

> Using [Morphological Analyzer JUMAN++ and Syntactic Analyzer KNP](https://github.com/ku-nlp/pyknp)

This module provides Japanese natural language processing capabilities using PyKNP (Python interface for JUMAN++ and KNP).

## Overview

The NLP module enables:
- **Morphological Analysis**: Break down Japanese text into morphemes using JUMAN++
- **Ruby Annotation**: Automatically add furigana (reading aids) to kanji characters
- **Text Formatting**: Convert between halfwidth and fullwidth characters
- **HTML Integration**: Process HTML content and add ruby annotations to Japanese text

## Key Features

- Automatic kanji-to-hiragana annotation
- Character width detection and conversion
- HTML content processing with Beautiful Soup
- Interactive display support for Jupyter notebooks

## Installation Requirements

Install PyKNP and JUMAN++ for Japanese morphological analysis:

Install [JUMAN++](https://github.com/ku-nlp/jumanpp)

```bash
pip install pyknp
```

## Core Text Processing Functions

These functions handle Japanese text analysis and ruby annotation generation using JUMAN++ morphological analyzer.

## HTML Processing

Additional dependencies for HTML content processing:

In [ ]:
# exporti
from pyknp import Juman

## HTML Content Annotation

This function processes HTML content and adds ruby annotations to Japanese text within HTML elements.

In [ ]:
# export
def is_halfwidth(text):
    """
    Determine whether the text consists entirely of halfwidth characters.
    :param text: Input text string
    :return: True if all characters are halfwidth, otherwise False
    """
    return all('\u0020' <= char <= '\u007E' or '\uFF61' <= char <= '\uFF9F' for char in text)

def halfwidth_to_fullwidth(text):
    result = ''
    for char in text:
        code = ord(char)
        if code == ord(' '):
            result += '　'
        elif ord('!') <= code <= ord('~'):
            # Convert ASCII characters in the range 0x0021 to 0x007E
            result += chr(code + 0xFEE0)
        else:
            result += char
    return result

def annotate(text):
    juman = Juman()
    for line in text.split('\n'):
        if line == '':
            yield '\n'
            continue
        mrphs = juman.analysis(line).mrph_list()
        for mrph in mrphs:
            if mrph.midasi == '\\␣':
                yield ' '
                continue
            if is_halfwidth(mrph.midasi):
                yield mrph.midasi
                continue
            if mrph.midasi == mrph.yomi:
                yield mrph.midasi
                continue
            yield f'<ruby>{mrph.midasi}<rt>{mrph.yomi}</rt></ruby>'
        yield '\n'

```bash
pip install beautifulsoup4
pip install lxml
```

In [ ]:
# exporti
import bs4
from ipymock.nlp import annotate
from IPython.display import display, HTML

In [ ]:
# export
def annotate_html(content, interactive = False):
    # Parse the HTML content with BeautifulSoup
    soup = bs4.BeautifulSoup(content, 'lxml-xml')

    # Iterate through the div elements and process only the leaf nodes
    for div in soup.find_all('div'):  # Find all 'div' elements in the soup
        # Check if the div is a leaf node (contains only text)
        if div.find_all(True):  # Has child tags, so it's a leaf node
            continue
        try:
            answser = ''
            line = ''
            for word in annotate(div.get_text()):
                answser += word
                if word == '\n':
                    if interactive:
                        display(HTML(line))
                    line = ''
                else:
                    line += word
            # Replace the content of the div with parsed HTML
            new_content = bs4.BeautifulSoup(answser, 'html.parser')
            div.clear()  # Clear the original content
            div.append(new_content)  # Append the new parsed content
        except Exception as e:
            print(f'Error processing content: {e}')
            continue

    return str(soup)  # Convert the soup back to string